Getting Started with Arcpy: 

In [ ]:
#Import module
import arcpy

#Setting up the workspace 
#Establishes the main path / source of files that are going to be used
arcpy.env.workspace = r'C:\Users\carvalhp\Desktop\GIS_Programming\Lab3\Lab3\Lecture3DemoExercises\spatial\vector\ExampleData.gdb\Iowa'

#Using an Arcpy function that lists all the feature classes in a geodatase
#Returns a list
fcList = arcpy.ListFeatureClasses(feature_type='Point')

#Prints items 
for theFC in fcList:
    
    print("The name of the feature class is " + theFC)
    
    

In [ ]:
#Clipping

#4 set a variable to reference the full file path of the clipping feature class
bh_boundary = r'C:\....'

#5 set a variable to reference the full path to hold output
output_local =  r'C:\.....'

for theFc in fcList:
        
    clipFc = arcpy.Clip_analysis(fc, bh_boundary, output_local + "\\..._" + fc)

In [ ]:
#Intersecting 

#Input
intFC1 = r'C:\...'
intFC2 = r'C:\...'

#Output
newFC = r'C:\Users\carvalhp\Desktop\GIS_Programming\Lab3\Lab3\Lecture3DemoExercises\output\_IowaCounty_DM_20200512.shp'

#put them in a list as Intersect tool requires a list (could be more than 2)
inputList = [intFC1, intFC2]


arcpy.Intersect_analysis(inputList, newFC)

In [ ]:
#Creating fields in the Attribute table
arcpy.AddField_management(newFC, "name", "TEXT","", "", 50)
arcpy.AddField_management(newFCtFC, "AreaAcres", "DOUBLE")

#Calculating value in the  field 
arcpy.management.CalculateField(newFC, "name", '!field1! + "_" + !field2!', "PYTHON3", '', "TEXT")

#Calculating geometry 
arcpy.CalculateGeometryAttributes_management(newFC, "AreaAcres AREA", '', "ACRES")

In [ ]:
#Search Cursor


referenceFC = r'C:\...'

#fields to look into
theFlds = ["xxxx", "yyyy", "www"]


#set up search cursor
with arcpy.da.SearchCursor(referenceFC, theFlds) as referenceCursor:
    
    
    for readRow in referenceCursor:
        
        #retrieve the value from "www" field based on index position
        tdsConc = readRow[2]
        
        #if condition...... 
        if tdsConc > 500:
            
            #report information
            print("....." + readRow[1])
            
            

In [ ]:
#Update Cursor



referenceFC = r'C:\...'

arcpy.AddField_management(referenceFC, "name", "Text", "", "", 8)

theFlds = ["xxxx", "yyyy", "www"]

with arcpy.da.UpdateCursor(referenceFC, theFlds) as updateCur:
    
    for updateRow in updateCur:
        
        tdsValue = updateRow[2] #index of field
        
        
        #Condition to populate the field created above
        if tdsValue is None:
            updateRow[3] = "Unknown"
        elif tdsValue > 500:
            updateRow[3] = "Yes"
        elif tdsValue <= 500:
            updateRow[3] = "No"
        
        updateCur.updateRow(updateRow)

In [ ]:
#Write Cursor

#variable to reference geodatabase stand-alone table to read
readTable = r'C:\.....'

#a list of fields to read from
readFlds = ["PlaceName", "ViewCount", "Longitude", "Latitude", "Link"]

#variable to reference feature class to write to
writeFC = r'C:\....'

#a list of fields to write to
writeFlds = ["SHAPE", "PlaceName", "ViewCount","Link"]

#setting up insert cursor 
writeCur = arcpy.da.InsertCursor(writeFC, writeFlds)

#set up search cursor 
with arcpy.da.SearchCursor(readTable, readFlds) as readCur:

    
    for readRow in readCur:
    
        #get values from the field by index of the field list
        thePlace = readRow[0]
        theCount = readRow[1] 
        theLong = readRow[2]
        theLat = readRow[3]
        theLink = readRow[4]
        
       
        #creating a point object fromt the lat, long values
        #write this point into the Shape field 
        newPoint = arcpy.Point(theLong, theLat)

        #insert the values into the table
        writeCur.insertRow([newPoint, thePlace, theCount, theLink])
        

#avoid file lock 
del writeCur

